In [17]:
import numpy as np
import numpy.linalg as la
def simplex_iteration(x, B, problem, verbose=False):
    """Perform one simplex iteration given 
    - basic feasible solution x
    - basis B
    
    It returns new x, new basis and termination flag (true/false)
    """
    A, b, c = problem['A'], problem['b'], problem['c']
    m, n = A.shape
    A_B = A[:, B]
    if verbose:
        print("x =", x)
        print("Basis Indicies/B =", B)
        print("c' * x =", c @ x)
        print("A_B =\n", A_B)
    
    # Compute reduced cost vector
    p = la.solve(A_B.T, c[B])
    c_bar = c - A.T @ p
    if verbose:
        print("Reduced costs/c_bar", c_bar)
    
    # Check optimality
    if np.all(c_bar >= -1e-10):
        if verbose:
            print("Optimal solution found!")
        return x, B, True
# Choose j such that c_bar < 0 (first one)
    j = np.where(c_bar < 0)[0][0]
    if verbose:
        print("j =", j)
    
    # Compute search direction d
    d = np.zeros(n)
    d[j] = 1
    d[B] = la.solve(A_B, -A[:, j])
    if verbose:
        print("New direction/d =", d, "\t( d_B =", d[B], ")")
    
    # Check for unboundedness
    if np.all(d >= 0):
        if verbose:
            print("Unbounded problem!")
        return None, None, True
    
    # Compute step length theta
    d_i = np.where(d[B] < 0)[0]
    theta = np.min(- x[B[d_i]] / d[B[d_i]])
    i = B[d_i[np.argmin(- x[B[d_i]] / d[B[d_i]])]]
    if verbose:
        print("i = ", i)
        print("Step length theta* = min_{i | d_i < 0} (-x_i/d_i) = min(", - x[B[d_i]] / d[B[d_i]], ") =", theta)
    
    # Compute next point
    x_next = x + theta * d
    
    if verbose:
        print("x_next = x + theta * d =", x_next)
    
    # Compute next basis
    B_next = B
    B_next[np.where(B == i)[0]] = j

  
    return x_next, B_next, False

def simplex_algorithm(x, B, problem, max_iter=1000, verbose=False):

    for k in range(max_iter):
        
        if verbose:
            print("\nIteration", k)
            
        x, B, end = simplex_iteration(x, B, problem, verbose=verbose)
        
        if end:
            break
            
       
    return x, B


In [18]:
problem = {
    'A': np.array([[2, 1, 1, 3], 
                   [1, 3, 1, 2]]),
    'b': np.array([5, 3]),
    'c': np.array([8, 8, 5, 9])
}

# Initial basic feasible solution
x = np.array([2.4, .2, 0, 0])

# Initial basis (indices of basic variables x3 and x4)
B = np.array([0, 1])  # Note: Python uses 0-based indexing

# Run simplex algorithm 
x1, B1 = simplex_algorithm(x, B, problem, max_iter = 1000, verbose = True)


# Output results
print("Optimal final x:", x1)
print("Optimal final basis:", B1)




Iteration 0
x = [2.4 0.2 0.  0. ]
Basis Indicies/B = [0 1]
c' * x = 20.8
A_B =
 [[2 1]
 [1 3]]
Reduced costs/c_bar [ 0.   0.   0.2 -3.8]
j = 3
New direction/d = [-1.4 -0.2  0.   1. ] 	( d_B = [-1.4 -0.2] )
i =  1
Step length theta* = min_{i | d_i < 0} (-x_i/d_i) = min( [1.71428571 1.        ] ) = 1.0
x_next = x + theta * d = [1. 0. 0. 1.]

Iteration 1
x = [1. 0. 0. 1.]
Basis Indicies/B = [0 3]
c' * x = 17.0
A_B =
 [[2 3]
 [1 2]]
Reduced costs/c_bar [-3.55271368e-15  1.90000000e+01  4.00000000e+00 -1.77635684e-15]
Optimal solution found!
Optimal final x: [1. 0. 0. 1.]
Optimal final basis: [0 3]
